In [1]:
# Complete TFIM VQE Implementation
# Generated by QuantumForge V4 on 2025-08-17 03:05:57
# Problem: 4-qubit TFIM with J=1.0, h=1.0

from qiskit_algorithms import VQE
from qiskit_algorithms.optimizers import L_BFGS_B
from qiskit.primitives import StatevectorEstimator
from qiskit.quantum_info import SparsePauliOp
from qiskit import QuantumCircuit
from qiskit.circuit import Parameter
import numpy as np

print("🚀 TFIM VQE Ground State Energy Calculation")
print("=" * 50)

# === TFIM Model Parameters ===
num_qubits = 4
coupling_strength = 1.0  # J parameter
field_strength = 1.0        # h parameter

print(f"TFIM Model: {num_qubits} qubits, J={coupling_strength}, h={field_strength}")

# === Build TFIM Hamiltonian ===
def build_tfim_hamiltonian(n_qubits: int, J: float, h: float) -> SparsePauliOp:
    """
    Build TFIM Hamiltonian: H = -J * ∑ZZ - h * ∑X
    
    Args:
        n_qubits: Number of qubits
        J: Coupling strength
        h: Transverse field strength
        
    Returns:
        TFIM Hamiltonian as SparsePauliOp
    """
    pauli_terms = []
    
    # Transverse field terms (X gates)
    for i in range(n_qubits):
        pauli_string = 'I' * n_qubits
        pauli_string = pauli_string[:i] + 'X' + pauli_string[i+1:]
        pauli_terms.append((pauli_string, -h))
    
    # Ising coupling terms (ZZ gates)
    for i in range(n_qubits - 1):
        pauli_string = 'I' * n_qubits
        pauli_string = pauli_string[:i] + 'Z' + pauli_string[i+1:]
        pauli_string = pauli_string[:i+1] + 'Z' + pauli_string[i+2:]
        pauli_terms.append((pauli_string, -J))
    
    return SparsePauliOp.from_list(pauli_terms)

hamiltonian = build_tfim_hamiltonian(num_qubits, coupling_strength, field_strength)
print(f"Hamiltonian: {len(hamiltonian.paulis)} Pauli terms")

# === Create VQE Ansatz ===
def create_tfim_ansatz(n_qubits: int, depth: int) -> QuantumCircuit:
    """
    Create TFIM-specific ansatz with X rotations and ZZ entangling gates
    
    Args:
        n_qubits: Number of qubits
        depth: Circuit depth
        
    Returns:
        Parameterized quantum circuit
    """
    qc = QuantumCircuit(n_qubits)
    
    for layer in range(depth):
        # X rotation layer (transverse field related)
        for qubit in range(n_qubits):
            theta = Parameter(f'rx_{layer}_{qubit}')
            qc.rx(theta, qubit)
        
        # ZZ entangling layer (coupling related)
        if "linear" == "linear":
            for qubit in range(n_qubits - 1):
                phi = Parameter(f'rzz_{layer}_{qubit}')
                qc.cx(qubit, qubit + 1)
                qc.rz(phi, qubit + 1)
                qc.cx(qubit, qubit + 1)
    
    # Final X rotation layer
    for qubit in range(n_qubits):
        theta = Parameter(f'rx_final_{qubit}')
        qc.rx(theta, qubit)
    
    return qc

ansatz = create_tfim_ansatz(num_qubits, 2)
print(f"Ansatz: {ansatz.depth()} depth, {len(ansatz.parameters)} parameters")

# === Configure Optimizer ===
optimizer = L_BFGS_B()
print(f"Optimizer: L_BFGS_B")

# === Initialize Parameters ===
initial_point = np.random.uniform(0, 2*np.pi, len(ansatz.parameters))

# === Setup and Execute VQE ===
estimator = StatevectorEstimator()
vqe = VQE(estimator, ansatz, optimizer)

print("\nExecuting VQE algorithm...")
vqe_result = vqe.compute_minimum_eigenvalue(hamiltonian)
ground_state_energy = vqe_result.eigenvalue.real

# === Results ===
print("\n" + "=" * 50)
print("🎯 VQE Results:")
print(f"Ground State Energy: {ground_state_energy:.8f}")
print(f"Optimizer Evaluations: {vqe_result.cost_function_evals}")
print(f"Optimal Parameters: {len(vqe_result.optimal_parameters)} values")
print("=" * 50)

# Final result
print(f"\n✅ TFIM 4-qubit ground state energy: {ground_state_energy:.8f}")


🚀 TFIM VQE Ground State Energy Calculation
TFIM Model: 4 qubits, J=1.0, h=1.0
Hamiltonian: 7 Pauli terms
Ansatz: 18 depth, 18 parameters
Optimizer: L_BFGS_B

Executing VQE algorithm...

🎯 VQE Results:
Ground State Energy: -4.33962370
Optimizer Evaluations: 1444
Optimal Parameters: 18 values

✅ TFIM 4-qubit ground state energy: -4.33962370
